### Ctran Breadcrumb Analysis

#### Definitions

**reading**: A **_reading_** is a unique entry from a c-tran breadcrumb data collection source (e.g. a bus pushing data). A **_reading_** is composed of multiple fields and is considered well-formed if all fields are present and all fields are of the correct datatype. Additionally, a **_reading_** is considered unique if it's well-formed and the set of it's field values are unique throughout the entirety of the data. In practice, each reading relates to each row in a database or dataframe.

At the time of writing, a **_reading_** contains these fields with these datatypes:

| Field name | EVENT_NO_TRIP | OPD_DATE | VEHICLE_ID | METERS  | ACT_TIME | GPS_LONGITUDE | GPS_LATITUDE |
|------------|---------------|----------|------------|---------|----------|---------------|--------------|
| Datatype   | integer       | date     | integer    | integer | integer  | float w/ precision 6 | float w/ precision 6|


Example:

| Field name | EVENT_NO_TRIP | OPD_DATE | VEHICLE_ID | METERS  | ACT_TIME | GPS_LONGITUDE | GPS_LATITUDE |
|------------|---------------|----------|------------|---------|----------|---------------|--------------|
| Values     | 152011646     | 24-FEB-20| 1776       | 77999   | 28558    | -122.579383   | 45.533608    |

**vehicle**: A **_vehicle_** is any datasource sending valid readings over time. Additionally, it must have a unique `VEHICLE_ID` which is sent within each reading. 

**path**: A **_path_** is a line connecting two points in 2D-space by the shortest path (i.e. as the crow flies). A point is defined as an `x`-`y` coordinate pair and its units are inconsequential -- as long as they're the same for botht the `x` and `y` coordintate. In practice, a point looks like a longitude-latitude coordinate pair, or an x-y coordinate pair with a known origin (e.g. the bottom left corner of Clark County). It's also important to note that, for the purposes of this project, the surface of any geodetic system (e.g. a sphere or the earth) is considered 2D-space. That is, the **_path_** between two points on the earth would be "curved" as it travels across the surface of it and not "through" it.

- route
- day
- iteration

#### Schema Definition

For the purposes of this notebook the schema is based off of pandas however, there there is a schema that defines a table within a postgresql database which is also defined here. The only notable difference is the datatype of the `GPS_LATITUDE` and `GPS_LONGITUDE` fields.


In [2]:
from glob import glob
import pandas as pd

# Grab all tsv data from the `data` directory
ALL_DATA_FILES = glob("../data/**/*.csv")

def load_data():
    ### Data columns are:
    ### "EVENT_NO_TRIP"	"OPD_DATE"	"VEHICLE_ID"	"METERS"	"ACT_TIME"	"GPS_LONGITUDE"	"GPS_LATITUDE"

    # Taken and modified from https://stackoverflow.com/a/21232849/4668680
    return pd.concat(
        [pd.read_csv(_file, parse_dates=[1]) for _file in ALL_DATA_FILES], ignore_index=True
    )

In [10]:
%timeit -n1 df = load_data()

8.13 s ± 85.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# print(df)
UNIQUE_TRIP_NOS = df["EVENT_NO_TRIP"].unique()
UNIQUE_DATES = df["OPD_DATE"].unique()
UNIQUE_VEHICLES = df["VEHICLE_ID"].unique()

#### Which dates are tracked, are these weekdays or weekend days?

In [173]:
from pandas.tseries.holiday import USFederalHolidayCalendar

# Test all dates are weekdays
unique_days = pd.Series(UNIQUE_DATES)
# print(unique_days.dt.day_name())

is_all_weekdays = pd.Series([(day != "Saturday" and day != "Sunday") for day in unique_days]).all()
print(is_all_weekdays)

True


In [174]:
df.OPD_DATE.agg(["min", "max"])

min   2020-02-24
max   2020-03-20
Name: OPD_DATE, dtype: datetime64[ns]

#### Are the same vehicles tracked each day or does it change from day to day?

In [175]:
date_group = df.groupby("OPD_DATE")

df2 = date_group.apply(lambda x: len(x["VEHICLE_ID"].unique()))

In [178]:
df2

OPD_DATE
2020-02-24    100
2020-02-25     95
2020-02-26    100
2020-02-27     97
2020-02-28     99
2020-03-02    100
2020-03-03     97
2020-03-04     97
2020-03-05     99
2020-03-06     98
2020-03-09    103
2020-03-10    102
2020-03-11    100
2020-03-12    102
2020-03-13    102
2020-03-16    102
2020-03-17    104
2020-03-18    104
2020-03-19    101
2020-03-20     99
dtype: int64

#### The ACT_TIME columns seems to represent the time at which the event occurred. what are its units?

In [179]:
# It's units appear to be seconds offset from midnight; this is the min/max time in hours
df.ACT_TIME.agg(["min", "max"]).div(60 * 60)

min     4.181111
max    25.683611
Name: ACT_TIME, dtype: float64

#### Relating to last: Can you convert this to datetime format?

In [184]:
# TODO: This would be more useful related to vehicle IDs
# TODO: This doesn't appear to maintain order
df["ACT_DTG"] = df.OPD_DATE + pd.to_timedelta(df.ACT_TIME, unit="s")
print(df.ACT_DTG)

0         2020-03-09 05:02:53
1         2020-03-09 05:02:58
2         2020-03-09 05:03:03
3         2020-03-09 05:03:08
4         2020-03-09 05:03:13
                  ...        
8623465   2020-03-20 18:35:05
8623466   2020-03-20 18:35:10
8623467   2020-03-20 18:35:15
8623468   2020-03-20 18:35:20
8623469   2020-03-20 18:35:25
Name: ACT_DTG, Length: 8623470, dtype: datetime64[ns]


#### Are the events spaced evenly in time or are the events more frequent at specific times of day (for a given vehicle)?

In [13]:
# What is the delta from reading to reading; is it always 5 seconds?

In [30]:
# What is the METERS field?
index = pd.Index(df.VEHICLE_ID)
# Meters with vehicle id index
meters_series = pd.Series(df.METERS, index=index)

monotonic_meters = pd.Series(meters_series.groupby(level=0).is_monotonic_increasing)
print(monotonic_meters.all())

Int64Index([   1776,    1776,    1776,    1776,    1776,    1776,    1776,
               1776,    1776,    1776,
            ...
            1298380, 1298380, 1298380, 1298380, 1298380, 1298380, 1298380,
            1298380, 1298380, 1298380],
           dtype='int64', name='VEHICLE_ID', length=8623470)
True


#### Which route has the largest birds-eye distance? The smallest distance?

In [185]:
index = pd.MultiIndex.from_frame(df[["OPD_DATE", "VEHICLE_ID"]])

# Index by unique vehicles per day
wanted_columns = ["GPS_LATITUDE", "GPS_LONGITUDE"]
lat_long_df = pd.DataFrame(
    df[wanted_columns].to_numpy(), columns=wanted_columns, index=index
)

# TODO: Why does vehicle id 1002 have NaN
# Intermediate step to min/max lat/long of each vehicle (across all days)
all_v_min_max = lat_long_df.groupby("VEHICLE_ID").agg(["min", "max"]).reindex(axis=1)
print(all_v_min_max)

# idx = pd.IndexSlice
# lat_mins = all_v_min_max.loc[:,["GPS_LATITUDE"]]
# maxs = all_v_min_max.loc[:,idx[:,"max"]]
# print(lat_mins)

           GPS_LATITUDE            GPS_LONGITUDE            
                    min        max           min         max
VEHICLE_ID                                                  
1002                NaN        NaN           NaN         NaN
1776          45.494440  45.736335   -122.689590 -122.503760
2215          45.494420  45.721972   -122.689602 -122.503748
2218          45.494393  45.736378   -122.689605 -122.503738
2220          45.494432  45.780962   -122.702240 -122.503760
...                 ...        ...           ...         ...
1254260       45.494432  45.722098   -122.702247 -122.503723
1254280       45.494422  45.721990   -122.689585 -122.503735
1254282       45.494375  45.724643   -122.689592 -122.503718
1254300       45.494418  45.723113   -122.702272 -122.503745
1298380       45.505045  45.780973   -122.686907 -122.475703

[117 rows x 4 columns]


In [186]:
# TODO: Use datetimes instead of strings for date comparisons
for bus_id in UNIQUE_VEHICLES:
    assert df[
        (df.OPD_DATE == "20-MAR-20") & (df.VEHICLE_ID == bus_id)
    ].METERS.is_monotonic, f"{bus_id} METERS is not monotonically increasing"

In [35]:
import geopandas
from shapely.geometry import Polygon
p1 = Polygon([(0, 0), (1, 0), (1, 1)])
p2 = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])
p3 = Polygon([(2, 0), (3, 0), (3, 1), (2, 1)])
g = geopandas.GeoSeries([p1, p2, p3])
g

0    POLYGON ((0.00000 0.00000, 1.00000 0.00000, 1....
1    POLYGON ((0.00000 0.00000, 1.00000 0.00000, 1....
2    POLYGON ((2.00000 0.00000, 3.00000 0.00000, 3....
dtype: geometry